In [1]:
!wget -O AnyLengthBertModelAndUtils.py https://raw.githubusercontent.com/DominicJW/AnyLengthBert/main/AnyLengthBertModelAndUtils.py
!wget -O AnyLengthBert.pt https://www.dropbox.com/scl/fi/u1m76jo7ai2vd5kbk1kym/AnyLengthBert.pt?rlkey=qhdw3f5yqw8qzmfslbz795qqx&st=wixhnsh2&dl=1
!pip install datasets
drive_path = "./"

--2024-04-23 01:55:07--  https://raw.githubusercontent.com/DominicJW/AnyLengthBert/main/AnyLengthBertModelAndUtils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12727 (12K) [text/plain]
Saving to: ‘AnyLengthBertModelAndUtils.py’

AnyLengthBertModelA 100%[===================>]  12.43K  --.-KB/s    in 0s      

2024-04-23 01:55:07 (91.5 MB/s) - ‘AnyLengthBertModelAndUtils.py’ saved [12727/12727]

--2024-04-23 01:55:07--  https://www.dropbox.com/scl/fi/u1m76jo7ai2vd5kbk1kym/AnyLengthBert.pt?rlkey=qhdw3f5yqw8qzmfslbz795qqx
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc

In [2]:
from AnyLengthBertModelAndUtils import my_collate, CustomDataset, set_seed, evaluate_loss, SentencePairClassifier, test_prediction, get_probs_from_logits
from  AnyLengthBertModelAndUtils import MaskedGlobalAvgPool1d, MaskedGlobalMaxPool1d
import torch
from torch.utils.data import DataLoader
import pandas as pd
import os
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef,accuracy_score, confusion_matrix



In [3]:
from google.colab import files
uploaded = files.upload()


Saving test.csv to test.csv


In [4]:
input_file = "test.csv"
output_preds_file = "Group_27_C.csv"
model_file = 'AnyLengthBert.pt'

df_test = pd.read_csv(input_file)

For AnyLengthBert

In [5]:
bert_model = 'bert-base-uncased'
maxlen = 128
bs = 32
test_set = CustomDataset(df_test,bert_model,maxlen,with_labels=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
path_to_output_probs_file = "probabilities"
test_loader = DataLoader(test_set, batch_size=bs, num_workers=5,collate_fn=my_collate)
model = torch.load(model_file)

model.eval()
print("Predicting on test data...")
test_prediction(net=model, device=device, dataloader=test_loader, with_labels=False,  # set the with_labels parameter to False if your want to get predictions on a dataset without labels
                result_file=path_to_output_probs_file)
print()
print("Probabilities are available in : {}".format(path_to_output_probs_file))

probs_test = pd.read_csv(path_to_output_probs_file, header=None)[0]
threshold = 0.5
preds_test=(probs_test>=threshold).astype('uint8')
out_df = pd.DataFrame(columns = ["prediction"])
out_df["prediction"] = preds_test
out_df.to_csv(output_preds_file,index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Predicting on test data...


  0%|          | 0/147 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 147/147 [00:15<00:00,  9.77it/s]


Probabilities are available in : probabilities


If labels are included compute performance metrics